# Stratification difference between the base run and the 10% increase run

CMFW May 2025

McFLURRIE project, ECCO Summer School, Pacific Grove CA

Notebook creates gifs and frames of the difference between the base run and an increased-runoff stratification in the upper 20m of ECCOv4r5 daily output. These plots are focused around the Arctic.

## Housekeeping and set-up

In [19]:
# -------------------
# Import Packages
# -------------------

# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys
# identify user's home directory
user_home_dir = expanduser('~')
# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

# Generic Packages
import os
import glob
import re
import cmocean
import cmocean
from dask.distributed import Client
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import zarr

In [20]:
# --------------------------------------------------------------
# Function to convert from iteration number (ECCO) to datetime
# Created by Mike Wood
# --------------------------------------------------------------

from datetime import timedelta, datetime

def date_to_iter_number(date,seconds_per_iter = 3600):
    total_seconds = (date-datetime(1992,1,1)).total_seconds()
    iter_number = total_seconds/seconds_per_iter
    return(iter_number)

def iter_number_to_date(iter_number,seconds_per_iter=3600):
    total_seconds = iter_number*seconds_per_iter
    date = datetime(1992,1,1) + timedelta(seconds=total_seconds)
    return(date)

In [21]:
# --------------------------------------
# Begin a dask client for ease + speed
# --------------------------------------

from dask.distributed import Client

#  connect to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:44067")
client.ncores
client.restart()

In [22]:
# ---------------------------
# Import the model geometry
# ---------------------------

# Your path may vary!
geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

In [23]:
# ---------------------------------------------------
# Check dates -- can change to desired year 
# ---------------------------------------------------

oct_2019_start = date_to_iter_number(datetime(2019,10,1))
dec_2019_end = date_to_iter_number(datetime(2019,12,31))

In [24]:
# -----------------------------------------
# Load in files for your year -- base run
# -----------------------------------------

# Choose whichever time span is interesting (be wary of memory!)
# I chose October, November and December 2019

# ------------------
# First for salt
# ------------------

# Create path from which we pull data
input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your time range
start_num = oct_2019_start
end_num = dec_2019_end

# Initiate empty list for storing data
salt_DA_list = []

# Read in each file, one at a time
for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename);
            salt_test = np.where(geom.hFacC == 1, salt_test, np.nan);

            tile = range(0, 13)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list.append(salt_DA);

# Concatenate all salinity files
salt_OND2019_base = xr.concat(salt_DA_list, dim='time');

# ------------------
# Repeat for theta
# ------------------

# Create path from which we pull data
input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your time range
start_num = oct_2019_start
end_num = dec_2019_end

# Initiate empty list for storing data
theta_DA_list = []

# Read in each file, one at a time
for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(0, 13)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list.append(theta_DA);

# Concatenate all valid files
theta_OND2019_base = xr.concat(theta_DA_list, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000243288.data
load_binary_array: data a

In [25]:
# -----------------------------------------
# Load in files for your year -- 1.1x run
# -----------------------------------------

# ----------------
# First for salt
# ----------------

# Create path from which we pull data
input_dir = '/efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean/'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2019_start
end_num = dec_2019_end

# Initiate empty list for storing data
salt_DA_list = []

# Read in each file, one at a time
for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename);
            salt_test = np.where(geom.hFacC == 1, salt_test, np.nan);

            tile = range(0, 13)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list.append(salt_DA);

# Concatenate all files
salt_OND2019_tenper = xr.concat(salt_DA_list, dim='time');


# ----------------
# Repeat for theta
# ----------------

# Create path from which we pull data
input_dir = '/efs_ecco/cwilliam/tenpercent/diags/THETA_daily_mean/'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Empty list for data storage
theta_DA_list = []

# Load in each timestep at a time
for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(0, 13)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list.append(theta_DA);

# Concatenate all valid files
theta_OND2019_tenper = xr.concat(theta_DA_list, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean/SALT_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean/SALT_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent/diags/SALT_daily_mean/SALT_daily_mean.0000243288.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_co

In [26]:
# ---------------------------------------------
# Select out the upper 20m, and desired tiles
# ---------------------------------------------

salt_OND2019_base = salt_OND2019_base.isel(tile=6).where(geom.Z>-20)
theta_OND2019_base = theta_OND2019_base.isel(tile=6).where(geom.Z>-20)
salt_OND2019_tenper = salt_OND2019_tenper.isel(tile=6).where(geom.Z>-20)
theta_OND2019_tenper = theta_OND2019_tenper.isel(tile=6).where(geom.Z>-20)
Z_u20 = geom.Z.where(geom.Z>-20)

In [57]:
# ------------------------------------------------
# Calculate stratification using gsw -- base run
# ------------------------------------------------

import gsw

# Initialise empty array
N2_map_base = np.zeros((len(salt_OND2019_base.time),90,90))
# Create mask for land
hFacC_arct = geom.hFacC.isel(tile=6)[0,:,:]

# Get frequency for each time step
for t in range(len(salt_OND2019_base.time)):
    # conversion of practical salinity to absolute salinity
    SA = gsw.SA_from_SP(salt_OND2019_base.isel(time=t), Z_u20, geom.XC.isel(tile=6), geom.YC.isel(tile=6))

    # conversion of potential temperature (poTemp) to conservative temperature (CT)
    CT = gsw.CT_from_pt(SA, theta_OND2019_base.isel(time=t))

    # # calculate pressure (dbar)
    P = gsw.p_from_z(Z_u20, geom.YC.isel(tile=6))

    # Brunt Vaisala Frequency (N^2)
    # Buoyancy frequency-squared at pressure midpoints, 1/s^2. 
    # The shape along the pressure axis dimension is one less than that of the inputs. (Frequency N is in radians per second.)
    # Pressure at midpoints of p, dbar. The array shape matches N2.
    N2, p_mid = gsw.Nsquared(SA, CT, P, geom.YC.isel(tile=6))
    N2_base = np.nansum(N2, axis=0)
    N2_map_base[t,:,:] = np.where(hFacC_arct, N2_base, np.nan)

In [58]:
# ------------------------------------------------
# Calculate stratification using gsw -- 1.1x run
# ------------------------------------------------

import gsw

# Initialise empty array
N2_map_tenper = np.zeros((len(salt_OND2019_tenper.time),90,90))
# Create land mask
hFacC_arct = geom.hFacC.isel(tile=6)[0,:,:]

# Calculate stratification for each timestep
for t in range(len(salt_OND2019_tenper.time)):
    # conversion of practical salinity to absolute salinity
    SA = gsw.SA_from_SP(salt_OND2019_tenper.isel(time=t), Z_u20, geom.XC.isel(tile=6), geom.YC.isel(tile=6))

    # conversiotn of potential temperature (poTemp) to conservative temperature (CT)
    CT = gsw.CT_from_pt(SA, theta_OND2019_tenper.isel(time=t))

    # # calculate pressure (dbar)
    P = gsw.p_from_z(Z_u20, geom.YC.isel(tile=6))

    # Brunt Vaisala Frequency (N^2)
    # Buoyancy frequency-squared at pressure midpoints, 1/s^2. 
    # The shape along the pressure axis dimension is one less than that of the inputs. (Frequency N is in radians per second.)
    # Pressure at midpoints of p, dbar. The array shape matches N2.
    N2, p_mid = gsw.Nsquared(SA, CT, P, geom.YC.isel(tile=6))
    N2_tenper = np.nansum(N2, axis=0)
    N2_map_tenper[t,:,:] = np.where(hFacC_arct, N2_tenper, np.nan)

In [64]:
# ------------------------------------------------------------
# Plotting a gif of difference between the 1.1x and base run
# ------------------------------------------------------------

import imageio
from matplotlib.colors import LogNorm

# Create data difference array
data = N2_map_tenper - N2_map_base

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

# Create colormap with grey land
cmap = cmocean.cm.diff.copy()
cmap.set_bad(color='gray')

# Initialise empty directory to store files
filenames = []

# Loop through time steps
for t in range(np.shape(N2_map)[0]):
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(data[t,:,:], origin='lower', cmap=cmap, vmin=-0.5e-8, vmax=0.5e-8)
    ax.set_title(str(salt_OND2019_u20.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'strat_tenper_base_2019.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Option to remove the temporary files, or keep them
# Sometimes useful to flick through each file!
# To remove these files, uncomment the lines below:

# # Cleanup
# for filename in filenames:
#     os.remove(filename)
# os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_2426/1899639512.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to strat_tenper_base_2019.gif
